<a href="https://colab.research.google.com/github/vokulovskiy/Counting-people-at-the-gate/blob/main/%D0%9F%D0%BE%D0%B4%D1%81%D1%87%D0%B5%D1%82_%D0%BB%D1%8E%D0%B4%D0%B5%D0%B9_%D0%BD%D0%B0_%D0%BF%D1%80%D0%BE%D1%85%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9_%D0%BF%D0%BE_%D0%BF%D0%B5%D1%80%D0%B5%D1%81%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D1%8E_%D0%BF%D0%BE%D0%BB%D0%B8%D0%B3%D0%BE%D0%BD%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics
%pip install supervision
## Подключение гугл диска для сохранения модели
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Подсчет людей внутри зоны
import cv2
import random
from ultralytics import YOLO
import supervision as sv
from supervision.notebook.utils import show_frame_in_notebook
import numpy as np
from google.colab.patches import cv2_imshow
from IPython import display
from supervision import VideoInfo, VideoSink

%matplotlib inline
display.clear_output()

# Задаем координаты вершин полигона, который мы будем использовать для подсчета проходящих объектов
POLYGON_COORDS = []
#Camera1
POLYGON_COORDS.append(['Out', [391, 380], [450, 700], [850, 650], [838, 325]])
POLYGON_COORDS.append(['In',  [398, 996], [450, 700], [850, 650], [1263, 993]])
#Camera2
#POLYGON_COORDS.append(['Out', [190, 400], [240, 700], [600, 650], [500, 400]])
#POLYGON_COORDS.append(['In',  [150, 999], [240, 700], [600, 650], [700, 999]])
#Camera3
#POLYGON_COORDS.append(['Out', [350, 400], [380, 850], [780, 520], [700, 270]])
#POLYGON_COORDS.append(['In',  [400, 999], [380, 850], [780, 520], [1200, 999]])
polygons=len(POLYGON_COORDS)
persons = [0 for _ in range(polygons)] # счетчик входящих/выходящих
deep = 3 # на скольки кадрах запоминаем людей
path='/content/gdrive/MyDrive/models/'
file_in='camera1_02.mp4'
file_out='camera1_02_m.mp4'
video_info = VideoInfo.from_video_path(video_path=path+file_in)
# Создаем экземпляр модели YOLO
model = YOLO("yolov8m.pt")
#model = YOLO('/content/gdrive/MyDrive/models/yolov8s_mod.pt')  # load a custom model

def inPolygon(x, y, crd):  # Функция для определения лежит ли точка внутри многоугольника
    c = 0
    for i in range(len(crd)):
        if (((crd[i][1] <= y and y < crd[i - 1][1]) or (crd[i - 1][1] <= y and y < crd[i][1])) and
                (x > (crd[i - 1][0] - crd[i][0]) * (y - crd[i][1]) / (crd[i - 1][1] - crd[i][1]) + crd[i][
                    0])): c = 1 - c
    return c


def inLabel(p, lb,  sl):  # выдает список с координатами найденных объектов "р" во всех слоях во всех полигонах кроме "sl"
    ret = []
    for i in range(len(lb)):
        for k in range(len(lb[i])):
            if k != sl:
                for j in range(len(lb[i][k])):
                    if lb[i][k][j][0] == p:
                        ret.append([i, k, j])
    return ret

def print_labels_obj(s=''):
    print(s)
    for sl in range(deep):
        print('kadr', sl - deep + 1)
        # print(*[i for i in range(polygons)],sep='\t')
        maxp = max([len(labels_obj[sl][k]) for k in range(polygons)])
        for no in range(maxp):
            for pl in range(polygons):
                if no <= len(labels_obj[sl][pl]) - 1:
                    print(labels_obj[sl][pl][no], end='\t')
            print()

# Создаем объекты для работы с подсчетом объектов и аннотированием полигона и прямоугольных рамок вокруг объектов
#zone1 = sv.PolygonZone(polygon=np.array(POLYGON_COORDS[0][1:]), frame_resolution_wh=video_info.resolution_wh)
#zone_annotator1 = sv.PolygonZoneAnnotator(zone=zone1, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
#zone2 = sv.PolygonZone(polygon=np.array(POLYGON_COORDS[1][1:]), frame_resolution_wh=video_info.resolution_wh)
#zone_annotator2 = sv.PolygonZoneAnnotator(zone=zone2, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

box_annotator = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5
)
box_annotator_c = sv.BoxAnnotator(
    thickness=2,
    text_thickness=1,
    text_scale=0.5,
    color=sv.Color.white()
)
kadr=0
labels_obj = [[[[random.randint(100, 110) for j in range(1)] for i in range(2)] for k in range(polygons)] for l in range(deep)]
# 1 Глубина - сколько кадров храним данные
# 2 количество полигонов
# 3 количество объектов внутри полигона
# 4 Метка, х, у

#Запись результата 
with VideoSink(target_path=path+file_out, video_info=video_info) as s:
    # Итерируемся по кадрам видео
    for result in model.track(source=path+file_in, show=True, stream=True, agnostic_nms=True, classes=0):#tracker="bytetrack.yaml",tracker="botsort.yaml"
        kadr+=1
        # Получаем текущий кадр и результаты детекции
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Присваиваем каждому объекту ID, чтобы отслеживать его в последующих кадрах
        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)
        
        # Оставляем только объекты, соответствующие нужным классам
        detections = detections[(detections.class_id == 0)]


        # Формируем метки для объектов
        labels = [
            f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]

        # Аннотируем прямоугольными рамками объекты на кадре
        frame = box_annotator.annotate(
            scene=frame, 
            detections=detections,
            labels=labels
        )
        #Умеьшаем рамки для детектирования в n раз
        n=100
        if result.boxes.id is not None:
            for i in range(len(detections.xyxy)):
                x1=detections.xyxy[i][0]
                y1=detections.xyxy[i][1]
                x2=detections.xyxy[i][2]
                y2=detections.xyxy[i][3]
                detections.xyxy[i][0]=x1+(x2-x1)/2-(x2-x1)/n
                detections.xyxy[i][1]=y1+(y2-y1)/2-(y2-y1)/n
                detections.xyxy[i][2]=x1+(x2-x1)/2+(x2-x1)/n
                detections.xyxy[i][3]=y1+(y2-y1)/2+(y2-y1)/n

        # Calculate
        labels_obj.pop(0) # Удаляем верхний слой
        labels_obj.append([])
        if detections.tracker_id is not None:
            for pl in range(polygons): #добавляем метки в полигоны
                labels_obj[deep-1].append([])
                cnt = len(detections.xyxy) #сколько объектов найдено на кадре
                for pers in range(cnt):
                    x = detections.xyxy[pers][0]
                    y = detections.xyxy[pers][1]
                    p = detections.tracker_id[pers]
                    if inPolygon(x, y, POLYGON_COORDS[pl][1:]):
                        #print(p, x, y, 'In Poligon ',pl)
                        r = inLabel(p, labels_obj, pl)
                        if len(r) > 0:  # был ли объект в другом полигоне, если да, то удаляем его и прибавляем счетчик
                            #print(p, 'in other polygons', r)
                            for rr in r:
                                labels_obj[rr[0]][rr[1]].pop(rr[2])
                            persons[pl] += 1
                        else:
                            labels_obj[deep - 1][pl].append([p, x, y]) # добавляем координаты объекта в полигон
            #print(persons)
            #print_labels_obj('Kadr '+str(kadr))

        # text
        frame = sv.draw.utils.draw_text(scene=frame, text='Out:'+str(persons[0]), text_anchor=sv.Point(x=150, y=200),text_color=sv.Color.white(),text_scale=1)
        frame = sv.draw.utils.draw_text(scene=frame, text=' In:'+str(persons[1]), text_anchor=sv.Point(x=150, y=250),text_color=sv.Color.white(),text_scale=1)
        # Рисуем прямоугольные рамки центров объектов на кадре
        frame = box_annotator_c.annotate(
            scene=frame, 
            detections=detections,
            labels=None, skip_label=True
        )

        # Подсчитываем количество объектов, пересекающих полигон
        #zone1.trigger(detections=detections)
        # Подсчитываем количество объектов, пересекающих полигон
        #zone2.trigger(detections=detections)

        # Аннотируем полигон на кадре
        #frame = zone_annotator1.annotate(scene=frame)
        #frame = zone_annotator2.annotate(scene=frame)
        frame = sv.draw.utils.draw_polygon(scene=frame, polygon=np.array(POLYGON_COORDS[0][1:]), color=sv.Color.white(),thickness=2)
        frame = sv.draw.utils.draw_polygon(scene=frame, polygon=np.array(POLYGON_COORDS[1][1:]), color=sv.Color.white(),thickness=2)
        
        # Отображаем кадр
        s.write_frame(frame=frame)
        #sv.show_frame_in_notebook(frame, (16, 16))
        #cv2_imshow(frame)
        # запись видео в файл
        #out.write(frame)
        # Отображаем кадр
        #cv2.imshow("yolov8", frame)

        # Выходим из цикла, если пользователь нажал клавишу ESC
        if (cv2.waitKey(30) == 27):
            break
        #if kadr>5:
            #break
    #out.release()



In [ ]:
Результаты
	        Out/In      	Out/In   	Out/In
	        реальные    	yolo8m   	yolo8s обученная	 
Camera1			            36/33		  41/38	
Camera2	    33/26		    32/29		  31/25	  Перекрытие
Camera3	    11/72		    11/61		  12/70	  Не достаточно кадров, не удачная линия пересечения
